# Zadanie 2 - Branislav Pecher

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
pd.set_option('display.max_columns', 50)

In [ ]:
def normalization(data, shift, scale):
    return (np.array(data) - float(shift))/scale

## Exploratívna analýza

Najprv si potrebujeme načítať dáta do premennej.

In [ ]:
train_data = pd.read_csv("train.csv", sep='\t')

In [ ]:
train_data.head()

Dáta máme načítané tak si môžeme pozrieť či obsahujú nejaké NaN hodnoty.

In [ ]:
train_data.info()

In [ ]:
train_data.shape[0] - train_data.dropna().shape[0]

Vidíme, že žiadne NA hodnoty sa v trénovacích dátach nenachádzajú a teda máme o starosť menej.

Ešte si môžeme pozrieť ako sú jednotlivé dáta rozdelené.

In [ ]:
train_data.describe(include = 'all')

Pridám si ešte aj namerané výsledky pre trénovacie dáta aby som mohol porovnať aj tieto a následne trénovať.

In [ ]:
train_data["class"] = pd.read_csv("train_target.csv", names = ["class"])
train_data.head()

Urobím si histogramy pre jednotlivé stĺpce aby som odhalil, či sa mi tu niekde nachádza outlier, poprípade aby som zistil aký tvar majú dáta.

In [ ]:
for i in range(0, len(train_data.columns)):
    train_data.hist(column = train_data.columns.values[i], bins=100)

Z grafov sme zistli, že veľmi veľa parametrov je veľmi silno zaťažených na jednu hodnotu (power law), až do takej miery, že v niektorých prípadoch zvyšné hodnoty nie je možné z histogramu odčítať, a teda toto bude treba v ďalšej kapitole ošetriť. Zároveň sme zistili, že sa tu nachádza zopár outlierov a teda ich vyhodím. Pri niektorých grafoch nie je možné zistiť či sa tam nenachádzajú outlieri a teda k vyhadzovaniu outlierom sa ešte vrátim po transformácii.

Vidím, že veľké percento dát má zadanú hodnotu triedy 0 čo je nezaradená trieda z dát. Keďže je to tak veľmi naklonené na jednu stranu tak bude zrejme treba znížiť významnosť tejto triedy.

Z exploratívnej analýzy vidím, že žiadne NA hodnoty sa pri týchto dátach nenachádzajú. Tiež vidím, že väčšina parametrov má celkom zlé rozdelenie a je veľmi silno zaťažená na jendu hodnotu. Zároveň aj vidím, že pri predikcii si bude treba dávať pozor na váhu triedy 0, keďže má veľmi silné zastúpenie v trénovacích dátach a teda aby nám neovplyvňovala výsledky - aby sa všetko nepredikovalo ako trieda 0.

## Feature engineering

In [ ]:
from scipy.stats import boxcox 

Aby sme dáta reálne mohli použiť, tak ich treba najprv normalizovať. Taktiež vyhodím stĺpce, ktoré nám nedávajú žiadnu výpovednú hodnotu - tie čo majú iba jednu hodnotu.

In [ ]:
name_list = ['cli_pl_body', 'cli_cont_len', 'aggregated_sessions', 'net_samples', 'tcp_frag', 'tcp_ooo',
             'cli_tcp_ooo', 'cli_tcp_frag', 'cli_win_zero', 'cli_tcp_full', 'cli_pl_change', 'srv_tcp_ooo',
             'srv_tcp_frag', 'srv_win_zero', 'cli_tx_time', 'proxy', 'sp_healthscore', 'sp_req_duration',
             'sp_is_lat', 'sp_error']

In [ ]:
transformed_data = pd.DataFrame()
box_param = pd.DataFrame(data=None, columns=train_data.columns,index=range(0,1))
for word in list(train_data.columns.values):
    if word == 'class':
        transformed_data['class'] = train_data['class']
    elif word not in name_list:
        transformed_data[word], box_param[word][0] = boxcox(train_data[word] + 1)

transformed_data.head()
del train_data

In [ ]:
percentile = pd.DataFrame(data=None, columns=transformed_data.columns,index=range(0,2))
for word in list(transformed_data.columns.values):
    if word not in name_list and word != 'class':
        percentile[word][0] = np.percentile(transformed_data[word], 25)
        percentile[word][1] = np.percentile(transformed_data[word], 75)
        transformed_data[word] = normalization(transformed_data[word], np.percentile(transformed_data[word], 25), np.percentile(transformed_data[word], 75))
    transformed_data.hist(column = word, bins = 100)

Po normalizácií už sú dáta v lepšej podobe a vidím, že sa tu zopár outlyerov nachádza preto ich vyhodím. Za outlyera považuje každú hodnotu ktorá je väčšia ako trojnásobok štandardnej odchýlky.

In [ ]:
for word in list(transformed_data.columns.values):
    if word != 'class':
        mean = np.mean(transformed_data[word])
        std = np.std(transformed_data[word])
        transformed_data = transformed_data[transformed_data[word] < mean + 3*std]
        transformed_data = transformed_data[transformed_data[word] > mean - 3*std]

In [ ]:
for word in list(transformed_data.columns.values):
    transformed_data.hist(column = word, bins=100)

Z histogramov vidíme, že dáta by už mali byť očistené dáta od outlierov (dalo by sa s nimi urobiť niečo iné ale, keďže máme veľa dát tak som sa ich rozhodol odstrániť) a normalizované na normálny tvar a teda ich môžem začat používať. Keďže však výsledné triedy sú veľmi silno zatažené na jednu hodnotu (hodnota 0, čiže neklasifikované), rozhodol som sa vyhádzať 90% tých riadkov, v ktorých trieda je 0 aby som predišiel možnému zameraniu na túto triedu.

In [ ]:
array = []
for i in np.random.rand(len(transformed_data)):
    array.append(i > 0.9)
    
undersampled_data = transformed_data[(transformed_data['class'] != 0) | (array)]
del transformed_data

In [ ]:
undersampled_data.hist(column='class', bins=100)

Z histogramu môžeme vidieť, že po odstránení 90% zástupcov triedy 0, je už rozdelenie viac vyvážené a že zastúpenie triedy 0 je približne rovnaké ako aj ostatných.

## Vyber klasifikatorov

Rozhodol som sa vyskúšať zopár klasifikátorov a porovnať ich medzi sebou aby som mohol vybrať ten najlepší. Vyskúšal som kNN klasifikátor, Naivného Gausovského Bayesa, SVM a Decision Tree.

Najprv si ale budem musiet načítať a upraviť validačné dáta.

In [ ]:
valid_data = pd.read_csv("valid.csv", sep='\t')
trans_valid = pd.DataFrame()
for word in list(valid_data.columns.values):
    if word not in name_list:
        trans_valid[word] = boxcox(valid_data[word] + 1, lmbda=box_param[word][0])

del valid_data

In [ ]:
for word in list(trans_valid.columns.values):
    if word not in name_list:
        trans_valid[word] = normalization(trans_valid[word], percentile[word][0], percentile[word][1])

In [ ]:
valid_test = np.genfromtxt("valid_target.csv", delimiter="\n")

Ešte budem musieť rozdeliť trénovacie dáta na normálne na ktorých sa trénuje a target.

In [ ]:
X, y = undersampled_data.drop("class", 1), undersampled_data["class"]
del undersampled_data

Keď už mám dáta v pamäti, môžem porovnávať klasifikátori

Hint: Nasledujúce modely budú mať možnosť trénovania dvoma spôsobmi - jeden na jednorázové použitie a druhý na ukladanie natrénovaného modelu. Púšťať obidva nemá moc zmysel 

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

### Bayes

Prvý spôsob - jednorázové použitie

In [ ]:
bayes = GaussianNB().fit(X, y)

Druhý spôsob - ukladanie modelu

In [ ]:
from sklearn.externals import joblib
import os.path as pth

if pth.exists('bayes.pk1'):
    bayes = joblib.load('bayes.pk1')
else:
    bayes = GaussianNB().fit(X, y)
    joblib.dump(bayes, 'bayes.pk1')

Keď mám model natrénovaný tak môžem predikovať validačné dáta a zistiť úspešnosť predikcie.

In [ ]:
out_valid = bayes.predict(trans_valid)
print(np.count_nonzero(out_valid == valid_test)/len(valid_test) * 100)
out_valid.tofile('bayes.csv', sep='\n', format="%d")

Toto porovnanie však ráta s predikovaním aj triedy 0 a preto použijem na kontrolu aj priamo dodaný skript.

In [ ]:
%run eval.py valid_target.csv bayes.csv

Vidíme, že úspešnosť je dosť zlá a teda sám o sebe nie je Naivný Bayes moc použiteľný.

### KNN

Prvý spôsob - jednorázové použitie

In [ ]:
neighbour = KNeighborsClassifier(5).fit(X, y)

Druhý spôsob - ukladanie modelu

In [ ]:
from sklearn.externals import joblib
import os.path as pth

if pth.exists('neighbours.pk1'):
    neighbour = joblib.load('neighbours.pk1')
else:
    neighbour = KNeighborsClassifier(5).fit(X, y)
    joblib.dump(bayes, 'neighbours.pk1')

Keď mám model natrénovaný tak môžem predikovať validačné dáta a zistiť úspešnosť predikcie.

In [ ]:
out_valid = neighbour.predict(trans_valid)
print(np.count_nonzero(out_valid == valid_test)/len(valid_test) * 100)
out_valid.tofile('neighbour.csv', sep='\n', format="%d")

Toto porovnanie však ráta s predikovaním aj triedy 0 a preto použijem na kontrolu aj priamo dodaný skript.

In [ ]:
%run eval.py valid_target.csv neighbour.csv

Vidíme, že úspešnosť je už lepšia avšak rýchlosť predikcie je značne pomalá a teda tento model nie je moc dobrý na predikciu.

### SVM

Prvý spôsob - jednorázové použitie

In [ ]:
svm = svm.SVC().fit(X, y)

Druhý spôsob - ukladanie modelu

In [ ]:
from sklearn.externals import joblib
import os.path as pth

if pth.exists('svm.pk1'):
    svm = joblib.load('svm.pk1')
else:
    svm = svm.SVC().fit(X, y)
    joblib.dump(svm, 'svm.pk1')

Keď mám model natrénovaný tak môžem predikovať validačné dáta a zistiť úspešnosť predikcie.

In [ ]:
out_valid = svm.predict(trans_valid)
print(np.count_nonzero(out_valid == valid_test)/len(valid_test) * 100)
out_valid.tofile('svm.csv', sep='\n', format="%d")

Toto porovnanie však ráta s predikovaním aj triedy 0 a preto použijem na kontrolu aj priamo dodaný skript.

In [ ]:
%run eval.py valid_target.csv svm.csv

Vidíme, že úspešnosť je celkom dobrá, avšak model je absolútne nepoužiteľný kvôli rýchlosti predikcie.

### Decision Tree

Prvý spôsob - jednorázové použitie

In [ ]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=None, random_state=42).fit(X, y)

Druhý spôsob - ukladanie modelu

In [ ]:
from sklearn.externals import joblib
import os.path as pth

if pth.exists('tree.pk1'):
    tree = joblib.load('tree.pk1')
else:
    tree = DecisionTreeClassifier(criterion='entropy', max_depth=None, random_state=42).fit(X, y)
    joblib.dump(tree, 'tree.pk1')

Keď mám model natrénovaný tak môžem predikovať validačné dáta a zistiť úspešnosť predikcie.

In [ ]:
out_valid = tree.predict(trans_valid)
print(np.count_nonzero(out_valid == valid_test)/len(valid_test) * 100)
out_valid.tofile('tree.csv', sep='\n', format="%d")

Toto porovnanie však ráta s predikovaním aj triedy 0 a preto použijem na kontrolu aj priamo dodaný skript.

In [ ]:
%run eval.py valid_target.csv tree.csv

Vidíme, že úspešnosť stromčeku je najlepšia z vyskúšaných modelov a taktiež aj predikcia je veľmi rýchla avšak sám o sebe nemá moc vysokú úspešnosť a teda ho bude treba použiť v nejakom ensembli.

# Ensebmle

Vyskúšal som viacero Ensemble spôsobov (Voting, Bagging, RandomForest, Boosting) s rôznymi úspechmi a rýchlosťami predikcie. Nižšie však uvádzam iba 2 ktorých pomer rýchlosti a presnosti bol zo všetkých najlepší.

Aj pri Ensembly budem mať 2 spôsoby - jeden s ukladaním a jeden bez.

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Prvý spôsob - jednorázové použitie

In [ ]:
forest = RandomForestClassifier(n_estimators=50, max_depth=7, n_jobs=1).fit(X, y)

Druhý spôsob - ukladanie modelu

In [ ]:
from sklearn.externals import joblib
import os.path as pth

if pth.exists('forest.pk1'):
    forest = joblib.load('forest.pk1')
else:
    forest = RandomForestClassifier(n_estimators=50, max_depth=7, n_jobs=1).fit(X, y)
    joblib.dump(forest, 'forest.pk1')

Keď mám model natrénovaný tak môžem predikovať validačné dáta a zistiť úspešnosť predikcie.

In [ ]:
out_valid = forest.predict(trans_valid)
print(np.count_nonzero(out_valid == valid_test)/len(valid_test) * 100)
out_valid.tofile('forest.csv', sep='\n', format="%d")

Toto porovnanie však ráta s predikovaním aj triedy 0 a preto použijem na kontrolu aj priamo dodaný skript.

In [ ]:
%run eval.py valid_target.csv forest.csv

Vidím, že úspešnosť nie je moc dobrá a dala by sa zlepšiť úpravou parametrou. Avšak už pri tomto nastavení parametrov to ledva zvláda pamäťovo Random Forest a keď ich iba o trochu upravím (zvýšim počet/hĺbku) tak padá na nedostatok pamäte. Preto som sa rozhodol spraviť aj Bagging.

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

Skúšal som viac nastavení Bagging klasifikátoru a tento čo nasleduje sa ukázal, že dáva najlepšie výsledky.

Prvý spôsob - jednorázové použitie

In [ ]:
bagging_forest = BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy', max_depth=None,
                                                                             random_state=42),
                                       n_estimators=150, max_samples=0.5, max_features=0.5, random_state=42).fit(X, y)

Druhý spôsob - ukladanie modelu

In [ ]:
from sklearn.externals import joblib
import os.path as pth

if pth.exists('bagging_forest.pk1'):
    bagging_forest = joblib.load('bagging_forest.pk1')
    print('bagging_forest')
else:
    bagging_forest = BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy', max_depth=None,
                                                                             random_state=42),
                                       n_estimators=150, max_samples=0.5, max_features=0.5, random_state=42).fit(X, y)
    joblib.dump(bagging_forest, 'bagging_forest.pk1')

Keď mám model natrénovaný tak môžem predikovať validačné dáta a zistiť úspešnosť predikcie.

In [ ]:
out_valid = bagging_forest.predict(trans_valid)
print(np.count_nonzero(out_valid == valid_test)/len(valid_test) * 100)
out_valid.tofile('bagging_forest.csv', sep='\n', format="%d")

Toto porovnanie však ráta s predikovaním aj triedy 0 a preto použijem na kontrolu aj priamo dodaný skript.

In [ ]:
%run eval.py valid_target.csv bagging_forest.csv

Vidím, že takto nastavený Bagging má veľmi dobrú úspešnosť a rýchlosť predikcie a preto ho aj použijem pri predikcii testovacích dát.

# Predikcia finálnych hodnôt

Teraz si už môžem načítať testovacie dáta a vytvoriť finálny csv súbor s predikovanými triedami. Testovacie dáta si budem zase musieť najprv upraviť pred predikciou.

In [ ]:
test_data = pd.read_csv("test.csv", sep='\t')
trans_test = pd.DataFrame()
for word in list(test_data.columns.values):
    if word not in name_list:
        trans_test[word] = boxcox(test_data[word] + 1, lmbda=box_param[word][0])

del test_data

In [ ]:
for word in list(trans_test.columns.values):
    if word not in name_list:
        trans_test[word] = normalization(trans_test[word], percentile[word][0], percentile[word][1])

In [ ]:
out_test = bagging_forest.predict(trans_test)
out_test.tofile('test_out.csv', sep='\n', format="%d")